# **Data 266 Final Project**


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, classification_report
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
business_df = pd.read_json('drive/My Drive/yelp_dataset/yelp_academic_dataset_business.json', lines=True)
user_df = pd.read_json('drive/My Drive/yelp_dataset/yelp_academic_dataset_user.json', lines=True)
tip_df = pd.read_json('drive/My Drive/yelp_dataset/yelp_academic_dataset_tip.json', lines=True)
review_df = pd.read_json('drive/My Drive/yelp_dataset/yelp_academic_dataset_review.json', lines=True)
checkin_df = pd.read_json('drive/My Drive/yelp_dataset/yelp_academic_dataset_checkin.json', lines=True)

In [3]:
!pip install -q sentencepiece
!pip install -q transformers

import nltk
import nltk.corpus
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
from nltk.tokenize import sent_tokenize, regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.probability import FreqDist

!pip install -q evaluate
import evaluate
!pip install -q rouge_score
from pprint import pprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Data Preprocessing

In [4]:
review_df.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [5]:
business_df.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [6]:
business_df = business_df.dropna(subset=['categories'])
restaurants = business_df[business_df['categories'].str.contains("Restaurants")]
restaurants_30 = restaurants[restaurants['review_count'] > 30]
reviews_over30 = review_df[review_df['business_id'].isin(restaurants_30['business_id'])]

In [7]:
def count_words(text):
    return len(text.split())

reviewwords_over50 = reviews_over30[reviews_over30['text'].apply(count_words) > 50]

In [8]:
reviews = reviewwords_over50.sample(frac=0.001, random_state=8)
reviews.shape

(2849, 9)

In [9]:
reviews.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
5024219,8cN568WT-shM6O9FuQ1xhw,-TdnnNdi7xAW-xR7lcEXXQ,IWHdx0NhDKADkGOgXgOFKQ,5,0,0,0,"Every day during lunchtime, 100's of hungry Ph...",2014-03-17 08:25:28
1917606,Au9BGeYCyl5cdArU3_t07Q,iZBiPVPfMYxer1CPGyozXw,lKJeRJm7WejutsrXWhVLiw,5,0,0,0,This was the best barbecue I've had on the who...,2021-02-19 15:54:10
5034327,iW1KdI1pVJqc8yDtB33HPw,9nuv3DsQJFjApI3UgHvgxQ,vXhrovlPWEnPKCbZMcsZDA,3,2,0,0,A solid 3.5!\n\nHoney Bakery is a tiny tiny sh...,2012-04-15 20:28:05
4480085,urNqHhsnsxoZfTTOMfetjQ,ZCehpsQdCmhvLG8Db9BH6A,XHbaHX2uBGIn3uw37jl7Og,5,0,1,0,We love Hodak's I have been going there ever s...,2018-09-01 02:43:53
6803271,edW2UBhJy91B3HWP-3qadg,b1XnODTuSShymyphtICkNA,iGAYmldEBt7ze_dJrBtHQg,4,0,0,0,We got take out twice in a row for a couple of...,2020-07-30 19:36:24


In [10]:
reviews.groupby(['stars'])['review_id'].size().reset_index()

,stars,review_id
0,1,354
1,2,289
2,3,381
3,4,718
4,5,1107


In [11]:
aggregated_reviews = reviews.groupby(['business_id']).agg({
    'stars': 'mean',
    'text': ''.join,
    'business_id': 'count'
}).rename(columns={'business_id': 'num_reviews'}).reset_index()
aggregated_reviews = aggregated_reviews.sort_values('num_reviews', ascending=False)
aggregated_reviews.head(5)

,business_id,stars,text,num_reviews
1453,_ab50qdWOk0DdB6XOrBitw,4.500000,We have been coming here for years (even when ...,6
1106,RQAF6a0akMiot5lZZnMNNw,3.333333,"I was so disappointed after my visit here, and...",6
511,C9K3579SJgLPp0oAOM29wg,4.600000,Charming and authentic. The pulled pork sandwi...,5
93,1b5mnK8bMnnju_cvU65GqQ,4.000000,God bless murica! Very good. I continue to swi...,5
575,DkVFvHGyNBGuPMzI9Zdt1A,3.200000,It was pretty average specialty hot dogs. It m...,5


In [12]:
sample = aggregated_reviews[aggregated_reviews['num_reviews'] >= 3]
sample.shape

(63, 4)

In [13]:
sample['text'].iloc[0]

"We have been coming here for years (even when food was served on paper plates). It's not the fanciest resturant but great in taste! I always have the seafood etouffee with crawfish tales, the charbroiled oysters & bread pudding - the best food in town. This is a must stop restaurant for the French Quarters!This place is amazing! Make sure you get there early if you don't want to stand in line we ended up sitting at the bar where they shuck oyster and got to know the friendly staff! I highly recommend the charbroiled oysters even my husband loved them (he does not eat oysters) we ordered a few different appetizers instead of meals and had a few beers. Overall it was a great experience and we can't wait to go back!Heard this place was amazing and that it fills up quick so boyfriend and I decided to come right at 11am on a weekday during our visit to NOLA. The server who sat us at the bar was not that nice and seemed to be in a hurry (even though the restaurant was only half full) but we

# Extractive Summarization - Sumy (LSA)

In [14]:
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 70.6 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=6a3548343c185945a72f03133fe856a11633df61d75f62796e61f6c4219fbe41
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=397ab0d67322454e7d0291668d9fdefd5d27628e729a2a0cd1c218a208ffa92f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [15]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Input text to be summarized
input_text = "" + sample['text'].iloc[0]

# Parse the input text
parser = PlaintextParser.from_string(input_text, Tokenizer("english"))

# Create an LSA summarizer
summarizer = LsaSummarizer()

# Generate the summary
summary = summarizer(parser.document, sentences_count=3)

# Output the summary
print("\nSummary:")
for sentence in summary:
    print(sentence)


Summary:
We have been coming here for years (even when food was served on paper plates).
and don't let the long line turn you away - it goes pretty quickly.
The wait outside Acme Oyster House usually looks long, but it moves rather fast.


In [16]:
def generate_summary_lsa(input_text):
    # Parse the input text
    parser = PlaintextParser.from_string(input_text, Tokenizer("english"))

    # Create an LSA summarizer
    summarizer = LsaSummarizer()

    # Generate the summary
    summary = summarizer(parser.document, sentences_count=3)

    # Output the summary
    return " ".join(str(sentence) for sentence in summary)

# Extractive Summarization - BERT

In [17]:
!pip install bert-extractive-summarizer
from summarizer import Summarizer

In [18]:
# Input text to be summarized
input_text = sample['text'].iloc[3]

# Create a BERT extractive summarizer
summarizer = Summarizer()

# Generate the summary
summary = summarizer(input_text, num_sentences=2)

# Output the summary
print(summary)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


I continue to switch between the east nasty (biscuit, fried chicken and gravy) and the Lily (their version of French toast). This was really nice of her and we took her up on the offer, but I just wish this wasn't the case.


In [19]:
def generate_summary_bert(input_text):
    # Create a BERT summarizer
    summarizer = Summarizer()

    # Generate the summary
    summary = summarizer(input_text, num_sentences=2)

    # Output the summary
    return summary

# Abstractive Summarization - T5

In [20]:
import tensorflow as tf

In [21]:
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration

In [22]:
t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-base') #also t5-small and t5-large
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

t5_model.summary()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
t5_input_text = "summarize: " + sample['text'].iloc[0]
t5_inputs = t5_tokenizer([t5_input_text], max_length=1024, truncation=True, return_tensors="tf")

In [24]:
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                    num_beams=3,
                                    no_repeat_ngram_size=5,
                                    min_length=30,
                                    max_length=50)

summary = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in t5_summary_ids][0]
summary

'chargrilled oysters are the only good things at the Acme Oyster House . the wait outside the restaurant usually looks long, but it moves rather fast .'

In [25]:
def generate_summary_t5(input_text, num_beams=3, no_repeat_ngram_size=5, min_length=30, max_length=50):
    t5_input_text = "summarize: " + input_text
    t5_inputs = t5_tokenizer([t5_input_text], max_length=1024, truncation=True, return_tensors="tf")
    t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                    num_beams=3,
                                    no_repeat_ngram_size=5,
                                    min_length=30,
                                    max_length=50)

    summary = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in t5_summary_ids][0]
    return summary

# Abstractive Summarization - Pegasus

In [26]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

In [27]:
pegasus_input_text = sample['text'].iloc[0]

In [28]:
pmodel_inputs = ptokenizer(pegasus_input_text, max_length=1024, truncation=True, return_tensors="tf")

In [29]:
pmodel_summary_ids = pmodel.generate(pmodel_inputs['input_ids'],
                                    num_beams=3,
                                    no_repeat_ngram_size=5,
                                    min_length=30,
                                    max_length=50)

summary = [ptokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in pmodel_summary_ids][0]
summary

'My husband and I first went to the Acme char-grilled oysters when we were in our early 20s and we loved them so much that we went back a few more times.'

In [30]:
def generate_summary_pegasus(input_text, num_beams=3, no_repeat_ngram_size=5, min_length=30, max_length=50):
    pegasus_input_text = input_text
    pmodel_inputs = ptokenizer(pegasus_input_text, max_length=1024, truncation=True, return_tensors="tf")
    pmodel_summary_ids = pmodel.generate(pmodel_inputs['input_ids'],
                                    num_beams=3,
                                    no_repeat_ngram_size=5,
                                    min_length=30,
                                    max_length=50)

    summary = [ptokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in pmodel_summary_ids][0]
    return summary

# Apply Summarization Algorithms

In [33]:
sample['LSA_Summary'] = sample['text'].apply(generate_summary_lsa)
sample['BERT_Summary'] = sample['text'].apply(generate_summary_bert)
sample['T5_Summary'] = sample['text'].apply(generate_summary_t5)
sample['Pegasus_Summary'] = sample['text'].apply(generate_summary_pegasus)

<ipython-input-33-129e6c2a98e2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['LSA_Summary'] = sample['text'].apply(generate_summary_lsa)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'a

In [35]:
sample.to_csv('/content/drive/MyDrive/summaries.csv', index=False)
sample.head(5)

,business_id,stars,text,num_reviews,LSA_Summary,BERT_Summary,T5_Summary,Pegasus_Summary
1453,_ab50qdWOk0DdB6XOrBitw,4.500000,We have been coming here for years (even when ...,6,We have been coming here for years (even when ...,We have been coming here for years (even when ...,chargrilled oysters are the only good things a...,My husband and I first went to the Acme char-g...
1106,RQAF6a0akMiot5lZZnMNNw,3.333333,"I was so disappointed after my visit here, and...",6,"So, I ordered a sandwich and hoped it would be...","I was so disappointed after my visit here, and...",the wait at dalessandros for a cheesesteak was...,Dalesandgies is a famous cheesesteak joint in ...
511,C9K3579SJgLPp0oAOM29wg,4.600000,Charming and authentic. The pulled pork sandwi...,5,"The beans were a bit peppery to our liking, bu...",The pulled pork sandwich was fantastic --the p...,a patron of a local brewery recommended this p...,"The reviews are in, and here's what some of yo..."
93,1b5mnK8bMnnju_cvU65GqQ,4.000000,God bless murica! Very good. I continue to swi...,5,We came in for brunch since the other place my...,I continue to switch between the east nasty (b...,the east nasty and the lilly is a great breakf...,Tennessee Biscuit is a fried chicken and biscu...
575,DkVFvHGyNBGuPMzI9Zdt1A,3.200000,It was pretty average specialty hot dogs. It m...,5,"We moved inside, but hard to get seated since ...",It was pretty average specialty hot dogs. No b...,"i'm usually a big dat dog fan, but yesterday i...","We've been to Dog before, but this is the firs..."


# Modeling

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [36]:
X_train, X_test, y_train, y_test = train_test_split(sample[['LSA_Summary', 'BERT_Summary', 'T5_Summary', 'Pegasus_Summary']], sample['stars'], test_size=0.2, random_state=42)

In [62]:
summary_features = ['LSA_Summary', 'BERT_Summary', 'T5_Summary', 'Pegasus_Summary']
models = {}

for feature in summary_features:
    # TF-IDF vectorization for the current feature
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train[feature])
    X_test_tfidf = vectorizer.transform(X_test[feature])

    # Train a linear regression model for the current feature
    model = LinearRegression()
    model.fit(X_train_tfidf, y_train)

    # Make predictions
    predictions = model.predict(X_test_tfidf)

    # Evaluate the model's performance
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE) for {feature} (TF-IDF): {rmse}")

    # Store the model for future reference
    models[feature + "LR"] = model

predictions, y_test

Root Mean Squared Error (RMSE) for LSA_Summary (TF-IDF): 0.6513878867866714
Root Mean Squared Error (RMSE) for BERT_Summary (TF-IDF): 0.8208902647121357
Root Mean Squared Error (RMSE) for T5_Summary (TF-IDF): 0.6605558851145377
Root Mean Squared Error (RMSE) for Pegasus_Summary (TF-IDF): 0.9163062321650532


(array([3.70968816, 4.47996741, 3.39147132, 3.51899343, 4.03358619,
        3.62709715, 4.4950209 , 3.70949531, 3.62386887, 3.64561351,
        4.04012727, 3.47364622, 4.13716029]),
 1048    3.666667
 1356    2.333333
 1453    4.500000
 161     4.333333
 1282    3.600000
 269     4.333333
 1014    3.750000
 1976    3.250000
 945     5.000000
 1058    4.000000
 1105    5.000000
 773     3.500000
 495     4.000000
 Name: stars, dtype: float64)

In [74]:
common_indices = sample.index.isin(X_test.index)
sample_test = sample[common_indices]
sample_test['text'].loc[1048]

'This has become a place I visit at least once while visiting Clearwater FL. I had the salmon and it was great before. They have a special menu item added every day and today was the roasted duck. It was so good and the sauce that came with it was sweet but had a spicy kick to it! The breakfast looks like it would be awesome, I just can\'t get up that early! Try the Gelato shop next to them after-wards for a good treat!This place was recommended to us by valet at our hotel.  We said we wanted someplace with great food that was lively.  We ate "outside" under cover and not long after we arrived it started raining, then pouring.  Some of the rain dripped through, which was not a big deal, but there was a situation with a gentleman and his dog that had come in the entryway to take cover from the rain.  It ended up with the police being called and a fist fight.  We did want lively.  Our waitress was Irish and was really wonderful. The food was good, I didn\'t think it was great.  I had the

In [75]:
sample_test['LSA_Summary'].loc[1048]

"The breakfast looks like it would be awesome, I just can't get up that early! They could do with a few less tables out there.This was the best restaurant that we stopped at on our trip to Clearwater. We saw some local police officers eating dinner there which is always a good sign."

In [76]:
sample_test['BERT_Summary'].loc[1048]

"This has become a place I visit at least once while visiting Clearwater FL. Her rice and vegetables we're spectacular especially after a few days of fried food."

In [78]:
sample_test['T5_Summary'].loc[1048]

'the lobster ravioli had huge (2-3inch) pieces of lobster and the dual colored pasta . the crab cakes were overly breaded but still good .'

In [77]:
sample_test['Pegasus_Summary'].loc[1048]

"We had a great time in Clearwater, Florida, and the food was just what we were looking for, but the service wasn't as good as we would have liked."

In [52]:
for feature in summary_features:
    # TF-IDF vectorization for the current feature
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train[feature])
    X_test_tfidf = vectorizer.transform(X_test[feature])

    # Train a linear regression model for the current feature
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_tfidf, y_train)

    # Make predictions
    predictions = model.predict(X_test_tfidf)

    # Evaluate the model's performance
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE) for {feature} (TF-IDF): {rmse}")
    # Store the model for future reference
    models[feature + "RF"] = model

Root Mean Squared Error (RMSE) for LSA_Summary (TF-IDF): 0.7034392601604227
Root Mean Squared Error (RMSE) for BERT_Summary (TF-IDF): 0.7511709577002476
Root Mean Squared Error (RMSE) for T5_Summary (TF-IDF): 0.8079901320058331
Root Mean Squared Error (RMSE) for Pegasus_Summary (TF-IDF): 0.8808748152665847


In [53]:
from sklearn.svm import SVR

for feature in summary_features:
    # TF-IDF vectorization for the current feature
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train[feature])
    X_test_tfidf = vectorizer.transform(X_test[feature])

    # Train a linear regression model for the current feature
    model = SVR(kernel='linear')
    model.fit(X_train_tfidf, y_train)

    # Make predictions
    predictions = model.predict(X_test_tfidf)

    # Evaluate the model's performance
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE) for {feature} (TF-IDF): {rmse}")
    # Store the model for future reference
    models[feature + "SVR"] = model

Root Mean Squared Error (RMSE) for LSA_Summary (TF-IDF): 0.6578358328119454
Root Mean Squared Error (RMSE) for BERT_Summary (TF-IDF): 0.7951595008407744
Root Mean Squared Error (RMSE) for T5_Summary (TF-IDF): 0.6539892072625739
Root Mean Squared Error (RMSE) for Pegasus_Summary (TF-IDF): 0.8834053395292357
